In [21]:
import cv2


In [26]:
  im = cv2.imread("cards.jpg")
  gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
  #blur = cv2.GaussianBlur(gray,(1,1),1000)
  flag, thresh = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY)

  contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

  cv2.drawContours(thresh,contours,-1,(255,80,30),3)
  highlight = cv2.cvtColor(im,cv2.COLOR_GRAY2BGR)
  cv2.imshow("image", thresh)
  key = cv2.waitKey(0)